# 🚀 Customize and Deploy `Qwen/Qwen3-VL-2B-Instruct` on Amazon SageMaker AI

---

In this notebook, we explore **Qwen3-VL-2B-Instruct**, a ~2-billion-parameter vision-language instruction-tuned model from Alibaba’s Qwen3 “VL” series. You’ll learn how to fine-tune it with image + text instruction workflows, evaluate its vision-language output performance, and deploy it via SageMaker for multimodal assistant use-cases.

**What is Qwen3-VL-2B-Instruct?**
Qwen3-VL-2B-Instruct is the instruction-tuned vision-language variant of the Qwen3 family at ~2B parameters. The “VL” denotes support for both image and text input, while “Instruct” indicates that the model has been fine-tuned to follow human instructions. The Qwen3 family is released under the **Apache-2.0 license**.

---

**Key Specifications**

| Feature                   | Details                                                                               |
| ------------------------- | ------------------------------------------------------------------------------------- |
| **Parameters**            | ~2 billion                                                                            |
| **Modalities**            | Image + Text input → Text output                                                      |
| **Context / Window Size** | Large text context (e.g., tens of thousands of tokens) combined with vision input     |
| **Architecture**          | Dense Transformer backbone with vision encoder front-end and instruction-tuned output |
| **License**               | Apache-2.0                                                                            |

---

**Benchmarks & Behavior**

* The Qwen3-VL models demonstrate strong capabilities in vision-language reasoning, structured output generation, and instruction following.
* At ~2B size, Qwen3-VL-2B-Instruct offers an efficient footprint for multimodal tasks where modest compute is available.
* Ideal for tasks such as image-guided question answering, vision-augmented instruction following, and assistant workflows that take images + text prompts.


In [ ]:
%pip install -Uq "datasets==4.3.0" \
    "sagemaker==2.253.1"

In [ ]:
import boto3
import sagemaker
from sagemaker.session import Session
from sagemaker.session import get_execution_role
from nb_local_utils.helpers import (
    pretty_print_html,
    get_mlflow_server_arn, 
    get_tracking_server_uri
)

In [ ]:
region = boto3.Session().region_name

sess = Session(boto3.Session(region_name=region))

sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = get_execution_role()

In [ ]:
pretty_print_html(f"sagemaker role arn: {role}")

In [ ]:
pretty_print_html(f"sagemaker bucket: {sagemaker_session_bucket}")

In [ ]:
pretty_print_html(f"sagemaker session region: {sess.boto_region_name}")

## 01. Data Preparation for Supervised Fine-tuning

### [DoclingMatix](https://huggingface.co/datasets/HuggingFaceM4/DoclingMatix)

**DoclingMatix** is a large-scale, multimodal dataset designed for document-intelligence tasks involving vision + instruction-tuned structured output. Built by augmenting the original Docmatix dataset, DoclingMatix converts each document image and QA pair into an instruction-based format for document conversion or visual question answering.

**Data Format & Structure**:

* Distributed in **Parquet** and/or **JSONL** formats.
* Contains a single `train` split with approximately **1,270,911** samples.
* Each record includes:

  * `images` – one or more images of the document page(s)
  * `instruction` – a natural-language prompt specifying the desired conversion or QA task (e.g., “Convert this page into structured DocTag format” or “Answer the question about the document”)
  * `response` – the target output (structured text or answer)

**License**: Released under the terms indicated on the Hugging Face dataset page (please verify before use).

**Applications**:

The dataset can support a variety of document-intelligence tasks, including:

* Instruction-tuned document conversion (image + instruction → structured text)
* Document visual question answering (DocVQA) and document understanding workflows
* Training vision–language models that parse document layouts and generate structured outputs
* Fine-tuning multimodal assistants or agents for enterprises, knowledge work, and document automation


In [ ]:
import os
import io
import base64
from PIL import Image
import json
import pprint
from tqdm import tqdm
from sagemaker.s3 import S3Uploader
from datasets import Dataset, load_dataset
from sagemaker.modules.configs import InputData

In [ ]:
dataset_parent_path = os.path.join(os.getcwd(), "tmp_cache_local_dataset")
os.makedirs(dataset_parent_path, exist_ok=True)

**Preparing Your Dataset in `messages` format**

This section walks you through creating a conversation-style dataset—the required `messages` format—for directly training LLMs using SageMaker AI.

**What Is the `messages` Format?**

The `messages` format structures instances as chat-like exchanges, wrapping each conversation turn into a role-labeled JSON array. It’s widely used by frameworks like TRL.

Example entry:

```json
{
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." },
    { "role": "user", "content": "How do I bake sourdough?" },
    { "role": "assistant", "content": "First, you need to create a starter by..." }
  ]
}


In [ ]:
dataset_name = "pranavvmurthy26/DoclingMatix_500"
dataset_train = load_dataset(dataset_name, split="train[:50]")

In [ ]:
pprint.pp(dataset_train[0])

In [ ]:
pretty_print_html(f"total number of fine-tunable samples: {len(dataset_train)}")

In [ ]:
def pil_to_base64(pil_img, resize_perc=0.2):
    """Convert a PIL image to base64-encoded PNG string."""
    if not isinstance(pil_img, Image.Image):
        raise ValueError("Input must be a PIL Image.")
    new_size = [int(resize_perc * s) for s in pil_img.size]
    pil_img = pil_img.resize(new_size)
    buffer = io.BytesIO()
    pil_img.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")


def convert_to_messages_multimodal(row):
    """Explode a multi-turn document QA row into multiple single-turn multimodal conversations."""
    system_prompt = (
        "You are a multimodal document reasoning assistant. Given an image of a document page "
        "and an instruction in natural language, generate a structured text output or answer "
        "that follows the instruction precisely. Process the visual and textual content in the image "
        "and provide a concise, well-formatted result. Explain briefly how the document image was used."
    )

    # Encode all images as base64 strings once per row
    images = []
    img_list = row.get("images", [])
    if img_list:
        for img in img_list if isinstance(img_list, list) else [img_list]:
            if hasattr(img, "save"):
                b64_img = pil_to_base64(img)
                images.append({
                    "type": "image_url",
                    "image_url": {"url": f"data:image/png;base64,{b64_img}"}
                })

    # Prepare outputs
    for t in row.get("texts", []):
        user_text = t.get("user", "").strip()
        assistant_text = t.get("assistant", "").strip()

        # Skip empty entries
        if not user_text or not assistant_text:
            continue

        yield {
            "messages": [
                {"role": "system", "content": [{"type": "text", "text": system_prompt}]},
                {"role": "user", "content": images + [{"type": "text", "text": user_text}]},
                {"role": "assistant", "content": [{"type": "text", "text": assistant_text}]}
            ]
        }


def convert_to_messages(dataset):
    """Iteratively run conversion on multi-turn conversation and flatten to messages"""
    records = []
    # Unroll your generator for every dataset row
    for row in tqdm(
        dataset, 
        total=len(dataset), 
        desc="Converting to messages"
    ):
        for example in convert_to_messages_multimodal(row):
            records.append(example)
    # Convert list of dicts → Hugging Face Dataset and return
    return Dataset.from_list(records)

#### Prepare a `training` dataset and upload file to `s3`

In [ ]:
dataset_train = convert_to_messages(dataset_train)

In [ ]:
dataset_filename_train = os.path.join(dataset_parent_path, f"{dataset_name.replace('/', '--').replace('.', '-')}--train.jsonl")
dataset_train.to_json(dataset_filename_train, lines=True)

In [ ]:
data_s3_uri = f"s3://{sess.default_bucket()}/dataset/{dataset_name}"

uploaded_train_s3_uri = S3Uploader.upload(
    local_path=dataset_filename_train,
    desired_s3_uri=data_s3_uri
)
print(f"Uploaded:\n--Source: {dataset_filename_train}\n>\n--Destination:{uploaded_train_s3_uri}")

---
---
END OF LAB 1
--- 
---
---


## 02. Fine-Tune LLMs using SageMaker `Estimator`/`ModelTrainer`

In [ ]:
from jinja2 import Template

### Training using `PyTorch` SageMaker `ModelTrainer`

**Training Using `PyTorch` Estimator**
Leverages the official PyTorch SageMaker container to run a custom training script using the Accelerate and DeepSpeed libraries. This option is ideal for users who want full control over the training pipeline 

---
**Observability**: SageMaker AI has [SageMaker MLflow](https://docs.aws.amazon.com/sagemaker/latest/dg/mlflow.html) which enables you to accelerate generative AI by making it easier to track experiments and monitor performance of models and AI applications using a single tool.

You can choose to include MLflow as a part of your training workflow to track your model fine-tuning metrics in realtime by simply specifying a **mlflow** tracking arn.

Optionally you can also report to : **tensorboard**, **wandb**.

In [ ]:
# automate tracking server fetch
MLFLOW_TRACKING_SERVER_ARN = get_mlflow_server_arn()

if MLFLOW_TRACKING_SERVER_ARN:
    reports_to = "mlflow"
else:
    reports_to = "tensorboard"

In [ ]:
mlflow_link = get_tracking_server_uri(sess, MLFLOW_TRACKING_SERVER_ARN)
pretty_print_html(f'<a href="{mlflow_link}" target="_blank">🔗 [Click Me to Open MLflow] 🔗</a>')

In [ ]:
job_name = 'Qwen--Qwen3-VL-2B-Instruct'

In [ ]:
if MLFLOW_TRACKING_SERVER_ARN:
    training_env = {
        "MLFLOW_EXPERIMENT_NAME": f"oss-recipe-{job_name}",
        "MLFLOW_TAGS": '{"source.job": "sm-training-jobs", "source.type": "sft", "source.framework": "pytorch"}',
        "MLFLOW_TRACKING_URI": MLFLOW_TRACKING_SERVER_ARN,
        "PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:True"
    }
else:
    training_env = {
        "PYTORCH_CUDA_ALLOC_CONF": "expandable_segments:True"
    }

In [ ]:
# attention mechanism
attention_mechanism = "flash_attention_2" # eager | flash_attention_2 | kernels-community/vllm-flash-attn3
# data path inside the container
datapath = f"/opt/ml/input/data/training/{os.path.basename(dataset_filename_train)}"
# max seq lengths
max_seq_lengths = 512
# training epochs
epochs = 1
# training batch size
training_batch_size = 1
# run name
run_name = f"Qwen3-VL-2B-Instruct-peft-qlora-{dataset_name.replace('/', '--').replace('.', '-')}"

In [ ]:
yaml_template = """# Model arguments
model_name_or_path: Qwen/Qwen3-VL-2B-Instruct
processor_name_or_path: Qwen/Qwen3-VL-2B-Instruct
model_revision: main
torch_dtype: bfloat16
attn_implementation: {{ attention_mechanism }}
use_liger: false
bf16: true
tf32: false
output_dir: /opt/ml/checkpoints/Qwen/Qwen3-VL-2B-Instruct/peft-qlora/

# Dataset arguments
dataset_id_or_path: {{ datapath }}
max_seq_length: {{ max_seq_lengths }}
packing: false

# LoRA arguments
use_peft: true
load_in_4bit: true
lora_target_modules: ["q_proj", "k_proj", "v_proj", "o_proj"]
# lora_modules_to_save: ["lm_head", "embed_tokens"]
lora_r: 4
lora_alpha: 16
lora_dropout: 0.05

# Modality type
modality_type: "image"

# Training arguments
num_train_epochs: {{ epochs }}
per_device_train_batch_size: {{ training_batch_size }}
gradient_accumulation_steps: 1
gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: true
learning_rate: 1.0e-4 
lr_scheduler_type: cosine
warmup_ratio: 0.1
remove_unused_columns: false

# Logging arguments
logging_strategy: steps
logging_steps: 1
report_to:
- {{ reports_to }}
run_name: {{ run_name }}
save_strategy: "epoch"
seed: 42
"""

config_filename = "Qwen3-VL-2B-Instruct-vanilla-peft-qlora.yaml"

In [ ]:
# Render template and plug dynamic values
rendered_yaml = Template(yaml_template).render(
    attention_mechanism=attention_mechanism,
    datapath=datapath,
    max_seq_lengths=max_seq_lengths,
    epochs=epochs,
    training_batch_size=training_batch_size,
    reports_to=reports_to,
    run_name=run_name
)

# Save to file (optional)
with open(os.path.join("./sagemaker_code/hf_recipes/Qwen", config_filename), "w") as f:
    f.write(rendered_yaml)

In [ ]:
pretty_print_html(rendered_yaml)

#### Training strategy: `PeFT/LoRA`

In [ ]:
instance_type = "ml.g5.2xlarge"
instance_count = 1

image_uri = sagemaker.image_uris.retrieve(
    framework="pytorch",
    region=sess.boto_session.region_name,
    version="2.7.1",
    instance_type=instance_type,
    image_scope="training",
)

pretty_print_html(f"Running training on: {image_uri}")

In [ ]:
from sagemaker.modules.configs import (
    CheckpointConfig,
    Compute,
    OutputDataConfig,
    SourceCode,
    StoppingCondition,
)
from sagemaker.modules.train import ModelTrainer

args = [
    "--config",
    "hf_recipes/Qwen/Qwen3-VL-2B-Instruct-vanilla-peft-qlora.yaml",
]

# Define the script to be run
source_code = SourceCode(
    source_dir="./sagemaker_code",
    requirements="requirements.txt",
    command=f"bash sm_accelerate_train.sh {' '.join(args)}",
)

# Define the compute
compute_configs = Compute(
    instance_type=instance_type,
    instance_count=instance_count,
    keep_alive_period_in_seconds=0,
)


# define OutputDataConfig path
output_path = f"s3://{sagemaker_session_bucket}/{job_name}"

# Define the ModelTrainer
model_trainer = ModelTrainer(
    training_image=image_uri,
    source_code=source_code,
    base_job_name=job_name,
    compute=compute_configs,
    stopping_condition=StoppingCondition(max_runtime_in_seconds=18000),
    environment=training_env,
    output_data_config=OutputDataConfig(
        s3_output_path=output_path,
        compression_type="NONE",
    ),
    checkpoint_config=CheckpointConfig(
        s3_uri=output_path + "/checkpoint", local_path="/opt/ml/checkpoints"
    ),
)

In [ ]:
from sagemaker.modules.configs import InputData

# Pass the input data
train_input = InputData(
    channel_name="training",
    data_source=uploaded_train_s3_uri,
)

# Check input channels configured
data = [train_input]
data

In [ ]:
# fit or train
model_trainer.train(input_data_config=data, wait=False)

In [ ]:
full_training_job_name = model_trainer._latest_training_job.training_job_name
pretty_print_html(f"Your training job name 👉 {full_training_job_name}")

In [ ]:
%store full_training_job_name

#### Get Training Job Status

In [ ]:
from sagemaker.estimator import Estimator

In [ ]:
estimator_state = Estimator.attach(
    training_job_name=full_training_job_name
)

In [ ]:
## print training job log (uncomment before execution)
# estimator_state.logs()

In [ ]:
s3_model_data_uri = estimator_state.model_data
pretty_print_html(f"Fine-tuned model location: {s3_model_data_uri}")

---
---
END OF LAB 2
--- 
---
---